<a href="https://colab.research.google.com/github/vannguyenhoai43-hash/ecommerce-category-growth-analysis/blob/main/notebooks/01_overview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1.1 Phân tích tăng trưởng theo Level 1 ngành hàng

Tại bước này, dữ liệu được nhóm theo Level 1 ngành hàng để tính:
- Chênh lệch tuyệt đối ADO & GMV giữa 2 giai đoạn
- Tốc độ tăng trưởng tương đối

Mục tiêu là so sánh **mức độ đóng góp tăng trưởng** giữa các ngành hàng chính.

##Nhập data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
path = "/content/drive/MyDrive/python/Quick_Report/Data_by_Month/Sep/data_by_level2.csv"
df_source_lv2 = pd.read_csv(path)

In [5]:
df_cur_lv2 = df_source_lv2[df_source_lv2['year_month'] == '2024-09']

## Hàm tăng trưởng

In [7]:
# Hàm tính tăng trưởng
def _growth(df_cat):

  ado_m = df_cat['ADO_M'].sum()
  ado_m1 = df_cat['ADO_M_1'].sum()
  gmv_m = df_cat['AdGMV_M'].sum()
  gmv_m1 = df_cat['AdGMV_M_1'].sum()

  return {
  'diff_ado' : ado_m - ado_m1,
  'diff_gmv' : gmv_m - gmv_m1,
  'grow_ado' : (ado_m - ado_m1) / ado_m1  if ado_m1 != 0 else 0,
  'grow_gmv':(gmv_m - gmv_m1) / gmv_m1  if gmv_m1 != 0 else 0,
}

In [1]:
# hàm tăng trưởng nhiều level
def growth_by_mul_level(df, level_cols):
    results = []

    base_col = ['ADO_M','ADO_M_1','AdGMV_M','AdGMV_M_1']

    for level_values, sub_df in df.groupby(level_cols):

        # tính growth cho từng group
        res = _growth(sub_df)

        # Giữ lại base col
        for c in base_col:
          res[c]=sub_df[c].sum()

        # gắn lại từng level vào kết quả
        for col, val in zip(level_cols, level_values):
            res[col] = val

        results.append(res)

    return pd.DataFrame(results)

In [9]:
import matplotlib.pyplot as plt

def plot_gmv_ado_dual_chart(df):


    fig, axes = plt.subplots(1, 2, figsize=(14, 4), sharey=False)

    categories = ['Automotive', 'Tools & Home Improvement']

    for ax, cat in zip(axes, categories):
        data = df[df['level1_kpi_category'] == cat].sort_values('year_month')

        # Bar chart - GMV
        ax.bar(data['year_month'], data['AdGMV_M'], width=0.5, alpha=0.8)
        ax.set_title(f"{cat} - GMV & ADO")
        ax.set_xlabel("Year-Month")
        ax.set_ylabel("GMV")

        # Line chart - ADO (secondary axis)
        ax2 = ax.twinx()
        ax2.plot(data['year_month'], data['ADO_M'],color='darkorange', marker='o')
        ax2.set_ylabel("ADO")

        # X label xoay cho dễ đọc
        ax.tick_params(axis='x', rotation=45)

    plt.tight_layout()
    plt.show()

#Bảng tăng trưởng ngành hàng

In [8]:
# tăng trưởng theo level 1 + level 2
l1_L2_growth = growth_by_mul_level(df_cur_lv2, ['level1_kpi_category','level2_kpi_category'])
l1_L2_growth_sort = l1_L2_growth.sort_values('level1_kpi_category')
print(l1_L2_growth_sort)

      diff_ado     diff_gmv  grow_ado  grow_gmv         ADO_M       ADO_M_1  \
0   240.151542  1166.658283  0.236101  0.182195   1257.305894   1017.154352   
10 -217.429876 -2871.259945 -0.017937 -0.042728  11904.668096  12122.097972   
9  -278.323065 -3071.923313 -0.038126 -0.124010   7021.828669   7300.151734   
7   359.185292  3088.709561  0.207875  0.196891   2087.075546   1727.890254   
6   410.234619  1375.558008  0.126093  0.079120   3663.657544   3253.422925   
8     0.018279  -107.672716  0.033332 -0.130589      0.566667      0.548388   
4     9.610908  5077.185056  0.074821  0.420203    138.062586    128.451678   
3    56.939348 -1074.727565  0.036420 -0.138811   1620.329208   1563.389860   
2   -92.698572 -1305.599699 -0.115163 -0.219878    712.234388    804.932960   
1    80.495411  -606.060025  0.044013 -0.059055   1909.389789   1828.894378   
5   290.833185  2088.187566  0.280875  0.335255   1326.289140   1035.455955   
15 -159.927383  1376.460112 -0.039813  0.041144   38